In [27]:
import pandas as pd
from nervaluate import Evaluator


In [20]:
df = pd.read_csv('TASTEset.csv')
df['entities'] = df.ingredients_entities.apply(lambda x: [i['entity'] for i in eval(x) if i['type'] not in ['QUANTITY', 'UNIT']] )
df['NER_tags'] = df.ingredients_entities.apply(lambda x: [i['type'] for i in eval(x) if i['type'] not in ['QUANTITY', 'UNIT']] )

In [24]:
ENTITIES = ['FOOD', 'TASTE', 'COLOR', 'PROCESS', 'PHYSICAL_QUALITY', 'PART', 'PURPOSE']
LABEL2ID = {'0':0}
idx = 1
for entity in Entities:
    LABEL2ID[f'B-{entity}'] = idx
    idx += 1 
    LABEL2ID[f'I-{entity}'] = idx
    idx += 1 

In [28]:
def evaluate_predictions(true_entities, pred_entities):
    """
    :param true_entities: list of true entities
    :param pred_entities: list of predicted entities
    :param format: format of provided entities. If equal to 'bio', entities
    are expected of the following format: [[B-FOOD, I-FOOD, O, ...],
    [B-UNIT, ...]]. If equal to span, entities are expected of the following
    format: [[(0, 6, FOOD), (10, 15, PROCESS), ...], [(0, 2, UNIT), ...]]
    :return: metrics for the predicted entities
    """

    evaluator = Evaluator(true_entities, pred_entities, tags=ENTITIES,
                              loader="list")

    results, results_per_tag = evaluator.evaluate()

    results = results["strict"]

    for entity in results_per_tag.keys():
        results_per_tag[entity] = results_per_tag[entity]["strict"]

    results_per_tag["all"] = results
    return results_per_tag